In [2]:
import ctypes
#hllDll = ctypes.WinDLL("C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v10.1\\bin\\cudart64_101.dll")
#incase cudart64_101.dll is unable to be loaded
import logging
import os
import numpy as np
from medpy.io import load
import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow.keras import layers
#import keras
import sklearn
from scipy.ndimage import rotate
import scipy
import time
from sklearn import datasets, preprocessing, model_selection, metrics
import matplotlib.pyplot as plt
%matplotlib inline

logging.basicConfig(level = 'logging.INFO')


In [3]:
'''Confirming running on GPU'''

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

CUDA_VISIBLE_DEVICES = 0,1

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15428671211691862370
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15278828619054665779
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3057018470
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16121076866482762412
physical_device_desc: "device: 0, name: GeForce GTX 1650 with Max-Q Design, pci bus id: 0000:02:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 15182840219776266020
physical_device_desc: "device: XLA_GPU device"
]
Num GPUs Available:  1


In [4]:
'''Predetermined Functions'''
def Generic_image_generator(URL):
    image_data, image_header = load(URL)
    for iter in range(0,len(image_data[:][:])):
        yield image_data[iter]

def create_ID_dict(basepath): #currently only set to return CT and segmentation mask images, not CTA's
    '''basepath is location of file that is holding dataset, datasets should be deeper in the file tree'''
    logging.info("creating dataset location directory/dictionary from {}".format(basepath))
    dict = {}
    mask_list = []
    image_list = []
    for root, dirs, list_of_files in os.walk(basepath):
        for files in list_of_files:
            if "r.mhd" in files:
                mask_list.append(os.path.join(root,files))
            else: 
                pass
            
            if "cti.mhd" in files:
                image_list.append(os.path.join(root,files))
            else:
                pass
            
            
    dict.update({"image" : image_list })
    dict.update({"mask" : mask_list })
    logging.info("dictionary created")
    
    return dict

def image_preprocessing(array):
    logging.info("Beginning Preprocessing")
    
    logging.info("Rotating Image into axial plane")
    array = rotate(array, 90, axes=(0,2), reshape=True) #rotates to axial form sup to inf
    
    logging.info("Making image to uniform size")
    
    blank_array = np.full((1,512,512),-3024) #-3024 is the "background, out of zoom data point"
    
    while len(array) < 69:
        array = np.append(array, blank_array, axis = 0)
        array = np.append(blank_array, array, axis = 0)
        
    if len(array) == 69:
        array = np.append(array,blank_array, axis = 0)
    else:
        pass
    
    logging.info("Image Preprocessing is complete")
    
    return array
    
def image_generator(dict_obj, mask):
    
    generate_mask = False
    logging.info("creating image generator for {}. Images are Segmentation Masks? = {}.".format(dict_obj,mask))
    
    if mask == True:
        generate_mask = True
    
    if generate_mask == True:
        for path in dict_obj['mask']:
            logging.info("loading {} {} into generator".format("mask",os.path.basename(path)))
            image, header = load(path)
            image = image_preprocessing(image)
            logging.info("Generator created")
            yield image, header
            
    elif generate_mask == False:
        for path in dict_obj["image"]:
            logging.info("loading {} {} into generator".format("image",os.path.basename(path)))
            image, header  = load(path)
            image = image_preprocessing(image)
            logging.info("Generator created")
            yield image, header
    else:
        raise TypeError("Problem with inputs.")
        logging.error("Unable to load image into generator")
            
    #no normalization has been added yet. Will need to think about how I want to approach this.
    #from skimage.color import rgb2gray to make things grayscale.
            


In [ ]:
def batch_slice_generator(generator, batchsize=5):
    while True:
        batch_img= []
        
        for img in generator:
            for slice in img:
                
            

In [ ]:
"""Need to do"""
'''
- Do I need to make a second generator for batch size? (https://www.kaggle.com/abhmul/python-image-generator-tutorial) 

'''

In [5]:
'''Predetermined Variables'''
PATH_tr = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\DataSets\\orCAScore Dataset\\Filtered\\Training_Set\\Train\\'
PATH_val = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\DataSets\\orCAScore Dataset\\Filtered\\Training_Set\\Validation\\'
PATH_te = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\DataSets\\orCAScore Dataset\\Filtered\\Testing_Set\\'
Model_path = 'C:\\Users\\Andrew_Arbogast\\Desktop\\Codes\\UCAIDM\\CAIDM Project\\Coronary_Calcium\\Models\\'
model_vers = 'First_Attempt'
input_shape = (70,512,512,1)

In [6]:
''' Loading Data Sets '''
logging.info("loading datasets")

start_time = time.time()
tr_dict = create_ID_dict(PATH_tr)
val_dict = create_ID_dict(PATH_val)
te_dict = create_ID_dict(PATH_te)
end_time = time.time()

print("Dictionary Creation Took {} seconds".format(end_time-start_time))

start_time = time.time()
tr_img = image_generator(tr_dict,mask = False)
tr_mask = image_generator(tr_dict, mask = True)
val_img = image_generator(val_dict, mask = False)
val_mask = image_generator(val_dict, mask = True)
te_img = image_generator(te_dict, mask = False)
end_time = time.time()


print("Generators Creation Took {} sec".format(end_time-start_time))


Dictionary Creation Took 0.009000778198242188 seconds
Generators Creation Took 0.0 sec


In [7]:
''' Creating NN'''


if os.path.isdir(Model_path+model_vers)==True:
    logging.info("Loading Network")
    model=tfk.models.load_model("Model_path+model_vers")
    
else:
    logging.info("Creating Network")
#https://towardsdatascience.com/step-by-step-implementation-3d-convolutional-neural-network-in-keras-12efbdd7b130    
    model= tfk.Sequential()
    model.add(layers.Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=input_shape))
    model.add(layers.MaxPooling3D(pool_size=(3, 3, 3)))
    model.add(layers.BatchNormalization(center=True, scale=True))
    model.add(layers.Dropout(0.5))
    model.add(layers.Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(layers.MaxPooling3D(pool_size=(3, 3, 3)))
    model.add(layers.BatchNormalization(center=True, scale=True))
    model.add(layers.Dropout(0.5))
    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(layers.Dense(15, activation='relu', kernel_initializer='he_uniform'))
    model.add(layers.Dense(5, activation='softmax'))


'''Hyper Parameters'''
logging.info("Adding Hyperparameters")

optimizer=tfk.optimizers.Adadelta()
loss="categorical_crossentropy"
metrics = ['accuracy']
model.compile(optimizer = optimizer, loss= loss, metrics= metrics )


In [ ]:
"""Other, Not in Use Model attempts"""

   
''' For a 2D network'''
"""model= tfk.Sequential()
    model.add(Conv2D(64, kernel_size=3, stride=(1,1), activation=’relu’, input_shape=(28,28,1)))
    model.add(Conv2D(32, kernel_size=3,stride=(1,1), activation=’relu’))
    model.add(Flatten())
    model.add(Dense(10, activation=’softmax’))"""
''' For a 3D network'''
    #tf.keras.layers.Conv3D(filters, kernel_size, strides=(1, 1, 1), padding='valid', data_format=None,
    #dilation_rate=(1, 1, 1), activation=None, use_bias=True,
    #kernel_initializer='glorot_uniform', bias_initializer='zeros',
    #kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
    #kernel_constraint=None, bias_constraint=None, **kwargs)
'''
model= tfk.Sequential()
input_shape = (70,512,512,1)
model.add(layers.Conv3D(filters=64, kernel_size=3,strides=(1, 1, 1), activation='relu', input_shape=input_shape))
model.add(layers.Conv3D(filters=32, kernel_size=3,strides=(1, 1, 1), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))'''

In [10]:
"""Training NN"""
logging.info("Training Neural Network")
model.fit(
    tr_img, tr_mask, 
    batch_size=1, 
    epochs = 10,
    verbose=2
)
#model.save("First_Attempt_Model/Model")


ValueError: `y` argument is not supported when using python generator as input.